In [1]:
import cv2
import numpy as np

In [2]:
def is_close_boxes(box1, box2, min_distance):
    """
    두 박스의 거리가 좁은지 확인합니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
        min_distance: 최소 거리
    
    Returns:
        True if 두 박스의 거리가 좁으면, False otherwise
    """

    b1_x, b1_y, b1_w, b1_h = box1
    
    box1_center_x = (b1_x + b1_w) / 2
    box1_center_y = (b1_y + b1_h) / 2

    
    b2_x, b2_y, b2_w, b2_h = box2

    box2_center_x = (b2_x + b2_w) / 2
    box2_center_y = (b2_y + b2_h) / 2
    
    # 두 박스의 중심점 간의 거리를 계산합니다.
    distance = np.sqrt(
        ((box1_center_x - box1_center_y)**2) +
        ((box2_center_x - box2_center_y)**2)
    )

    print(distance, distance <= min_distance)

    return distance <= min_distance

In [3]:
def merge_boxes(box1, box2):
    """
    두 박스를 하나의 박스로 합칩니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
    
    Returns:
        합쳐진 박스
    """
    
    b1_x, b1_y, b1_w, b1_h = box1
    b2_x, b2_y, b2_w, b2_h = box2
    
    return (
        min(b1_x, b2_x), # X
        min(b1_y, b2_y), # Y
        max(b1_w, b2_w), # W
        max(b1_h, b2_h), # H
    )

In [4]:
def find_objects(image, min_distance):
	"""
	이미지에서 윤곽선이 들어가는 오브젝트를 찾아 박스로 표기하고, 박스간의 거리가 좁으면 하나의 박스로 합쳐줍니다.
	
	Args:
		image: PIL 이미지 객체
		min_distance: 박스간의 최소 거리
	
	Returns:
		박스로 표기된 이미지 (PIL 이미지 객체)
	"""
	
	# 이미지를 그레이스케일로 변환합니다.
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	
	# 가우시안 블러 적용으로 노이즈 제거
	blurred = cv2.GaussianBlur(gray, (5, 5), 0)

	# 객체보다 배경이 밝은 경우 이미지 반전
	gray = cv2.bitwise_not(blurred)
	
	# Canny 알고리즘을 사용하여 윤곽선을 추출합니다.
	_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

	thresh = cv2.erode(thresh, None, iterations=2)
	thresh = cv2.dilate(thresh, None, iterations=2)

	# 윤곽선을 추출합니다.
	contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	
	# 박스를 표기하고 거리를 계산합니다.
	boxes = []
	for contour in contours:
		x, y, w, h = cv2.boundingRect(contour)
		box = (x, y, x+w, y+h)
		boxes.append(box)
	
	print(boxes)
	
	
	# 거리가 좁은 박스를 하나의 박스로 합칩니다.
	merged_boxes = []
	for i in range(len(boxes)):
		merged_box = boxes[i]
		
		for j in range(i+1, len(boxes)):
			if is_close_boxes(merged_box, boxes[j], min_distance) == True:
				merged_box = merge_boxes(merged_box, boxes[j])
				merged_boxes.append(merged_box)
			
	print(merged_boxes)
	
	# 합쳐진 박스를 표기합니다.
	for box in merged_boxes:
		cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
	
	return image

In [5]:
# 이미지 불러오기
image = cv2.imread("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000002.png")

# 오브젝트 찾기
object_image = find_objects(image, min_distance=10000)

[(3492, 143, 3534, 166), (3512, 136, 3526, 146), (975, 131, 1011, 147), (3666, 129, 3702, 160), (3660, 115, 3696, 131), (1335, 113, 1393, 145), (3217, 112, 3222, 117), (3468, 111, 3531, 133), (2719, 107, 2761, 134), (2577, 104, 2624, 127), (947, 104, 1005, 130), (2279, 103, 2345, 114), (2603, 101, 2616, 107), (1835, 98, 1894, 136), (1125, 97, 1162, 147), (1549, 96, 1590, 129), (1970, 91, 2020, 126), (488, 90, 548, 125), (247, 90, 300, 140), (1319, 89, 1380, 102), (3376, 84, 3436, 159), (2046, 81, 2098, 128), (3357, 75, 3362, 80), (3363, 73, 3411, 119), (2367, 72, 2431, 126), (3091, 70, 3127, 107), (3217, 68, 3255, 95), (3470, 67, 3539, 116), (2858, 65, 2876, 73), (2559, 64, 2617, 91), (3226, 61, 3284, 141), (392, 61, 417, 112), (3601, 60, 3638, 130), (1107, 58, 1147, 95), (3140, 57, 3179, 135), (1620, 57, 1658, 112), (2282, 56, 2318, 90), (3653, 54, 3684, 110), (592, 54, 599, 59), (565, 54, 614, 113), (3373, 53, 3393, 67), (2724, 53, 2754, 97), (1674, 53, 1698, 134), (618, 53, 644, 116

In [6]:
len(object_image)

176

In [7]:
from time import time

# 결과 이미지 출력
cv2.imwrite(f'./image/None-{time()}.jpg', object_image)

True